### Database with Faulty Readers in Go [14 points]

As a reminder, common constructs in Go are:
- `x := E` for declaring `x` and assigning to it
- `x <- ch` for receiving over `ch`
- `ch <- E` for sending over `ch`
- `select {case ... : ... case ... : ... default ...}` for nondeterministic choice
- `for {...}` for an infinite loop
- `for i := 0; i < 10; i++ {...}` for a "for" loop
- `make(chan T)` for a synchronous channel of type `T`
- `make(chan T, C)` for an asynchronous channel of type `T` with capacity `C`
- `<- time.After(M * time.Millisecond)` for a message after `M` milliseconds

Consider following implementation of a "database" that stores only a single integer. Writers can update that integer by sending a new value over the `writes` channel and readers can read it by sending a reply channel over the `reads` channel. Initially the database only accepts the initial value over the `writes` channel.

In [1]:
%%writefile db.go
package main

import ("math/rand"; "time")

const R = 100 // number of readers
const W = 10 // number of writers

var done chan bool

func db(reads chan chan int, writes chan int) {
    state := <- writes
    for {
        select {
        case reply := <-reads: reply <- state
        case state = <-writes:
        }
    }
}
func main() {
    reads, writes, done := make(chan chan int), make(chan int), make(chan bool)
    go db(reads, writes)
    for r := 0; r < R; r++ { // create R readers 
        go func() {
            reply := make(chan int)
            for i := 0; i < 20; i++ { 
                reads <- reply // stops in one out of 10 cases
                if rand.Intn(10) == 0 {select{}} else {<- reply}
                time.Sleep(time.Millisecond)
            }
        }()
    }
    for w := 0; w < W; w++ { // create W writers
        go func() {
            for i := 0; i < 20; i++ {
                writes <- rand.Int() % 100
                time.Sleep(time.Millisecond)
            }
            done <- true
        }()
    }
    for k := 0; k < W; k++ {<- done} // wait for writers to terminate
}

Writing db.go


In [ ]:
!go run db.go

In the implementation above, readers may fail: a reader may send a request over the `reads` channel and then die, i.e. not accept a reply from the database. Here, in 1 out of 10 cases, a reader stops after sending a request. What happens to the database and all other readers and writers in that case? Note that the Go library function [`rand.Intn(n)`](https://golang.org/pkg/math/rand/#Intn) randomly generates a non-negative number less than `n`.  [1 point]

YOUR ANSWER HERE

We consider three different remedies. The first one is to use buffered channels (asynchronous communication). Which channels need to be buffered and what is the smallest capacity to keep the database running with the given readers and writers? Copy above code and make **minimal** modifications! Add the explanation as a comment and mark the modified lines. [2 points]

In [3]:
%%writefile db.go
package main

import ("math/rand"; "time")

const R = 100 // number of readers
const W = 10 // number of writers

var done chan bool

func db(reads chan chan int, writes chan int) {
    state := <- writes
    for {
        select {
        case reply := <-reads: reply <- state
        case state = <-writes:
        }
    }
}
func main() {
    reads, writes, done := make(chan chan int), make(chan int), make(chan bool)
    go db(reads, writes)
    for r := 0; r < R; r++ { // create R readers 
        go func() {
            reply := make(chan int, 1)
            for i := 0; i < 20; i++ { 
                reads <- reply // stops in one out of 10 cases
                if rand.Intn(10) == 0 {select{}} else {<- reply}
                time.Sleep(time.Millisecond)
            }
        }()
    }
    for w := 0; w < W; w++ { // create W writers
        go func() {
            for i := 0; i < 20; i++ {
                writes <- rand.Int() % 100
                time.Sleep(time.Millisecond)
            }
            done <- true
        }()
    }
    for k := 0; k < W; k++ {<- done} // wait for writers to terminate
}

Overwriting db.go


In [4]:
!go run db.go

The second remedy is to keep unbuffered channels, but on each read request, the database creates an auxiliary goroutine which sends the state over the reply channel, rather than the `db` goroutine sending it. In case the reader dies, the auxiliary goroutine will not be able to send, but the database will still be able to continue running. Copy the unmodified code above and make **minimal** modifications to `db` only! Explain in a comment which state of the database is being sent, e.g. the state at the time the request is received, the state at the time the reply is sent back, or some other state! Mark all the modified lines. [2 points]

In [ ]:
%%writefile db.go
package main

import ("math/rand"; "time")

const R = 100 // number of readers
const W = 10 // number of writers

var done chan bool

func db(reads chan chan int, writes chan int) {
    state := <- writes
    for {
        select {
        case reply := <-reads: 
            //EDIT: Run anonymous goroutine to send state to reply
            //The sent state is the current state at the time the reply is sent
            //EDITS START HERE
            go func() {
                reply <- state
            } ()
            //EDITS END HERE
        case state = <-writes::
        }
    }
}
func main() {
    reads, writes, done := make(chan chan int), make(chan int), make(chan bool)
    go db(reads, writes)
    for r := 0; r < R; r++ { // create R readers 
        go func() {
            reply := make(chan int)
            for i := 0; i < 20; i++ { 
                reads <- reply // stops in one out of 10 cases
                if rand.Intn(10) == 0 {select{}} else {<- reply}
                time.Sleep(time.Millisecond)
            }
        }()
    }
    for w := 0; w < W; w++ { // create W writers
        go func() {
            for i := 0; i < 20; i++ {
                writes <- rand.Int() % 100
                time.Sleep(time.Millisecond)
            }
            done <- true
        }()
    }
    for k := 0; k < W; k++ {<- done} // wait for writers to terminate
}

In [ ]:
!go run db.go

The third remedy is to use a timeout: the database will try for at most `10 ms` to send the reply back. If the reply cannot be sent to the requesting reader, the database simply continues. Copy the unmodified code above and make **minimal** modifications to `db` only! Mark all the modified lines. [3 points]

In [ ]:
# YOUR CODE HERE
raise NotImplementedError()

In [ ]:
!go run db.go

Discuss the merits of the three different remedies with respect to following aspects:
1. tolerance of the database with respect to "slow" readers, i.e. readers that may not be immediately ready to receive a reply
2. allowing multiple readers to read concurrently and to read concurrently with a writer
3. resource consumption in terms of additionally needed channels, goroutines, memory, etc.

Give brief answer to each of the three points! [3 points]

YOUR ANSWER HERE

In case there are both readers sending over `reads` and writers sending over `writes` to the database, how is the choice resolved in `select` statement of the database in Go? [1 point]

YOUR ANSWER HERE

Modify the database such that writers are given preference! Copy the unmodified code of `db` and make **minimal** modifications. Testing is not required, it is up to you to include test code if you like. To simplify testing, readers have been modified so as not to fail. _Hint:_ Use `select` with `default`. [2 points]

In [ ]:
%%writefile db.go
package main

import ("math/rand"; "time")

const R = 100 // number of readers
const W = 10 // number of writers

var done chan bool

# YOUR CODE HERE
raise NotImplementedError()
func main() {
    reads, writes, done := make(chan chan int), make(chan int), make(chan bool)
    go db(reads, writes)
    for r := 0; r < R; r++ { // create R readers
        go func() {
            reply := make(chan int)
            for i := 0; i < 20; i++ { 
                reads <- reply; <- reply
                time.Sleep(time.Millisecond)
            }
        }()
    }
    for w := 0; w < W; w++ { // create W writers
        go func() {
            for i := 0; i < 20; i++ {
                writes <- rand.Int() % 100
                time.Sleep(time.Millisecond)
            }
            done <- true
        }()
    }
    for k := 0; k < W; k++ {<- done} // wait for writers to terminate
}

In [ ]:
!go run db.go